# tutorial for this script

## tokenizer.py

In [1]:
import MeCab
import CaboCha

# NGワードがあれば追加
NG_WORDS=[
# "AAA","BBB"
]

In [2]:
def extract_word(text):
    """
    textを受け取り、
    ・内容語（名詞・動詞・形容詞）
    ・名詞句（名詞が続く）
    ・内容語が含まれるbi-gram
    を抽出し、listで返す
    """
    # 名詞句調査
    is_prefix = 0 # 接頭詞の発見フラグ
    is_prefix_and_noun = 0 # 接頭詞に続く名詞の発見フラグ
    cnt_head_noun = 0 # 名詞句の構成単語が何単語含まれるか
    tmp_noun_phrase = "" # 名詞句の一時保存
    # 内容語のbigram調査
    is_main_word = 0 # 内容語であるか
    is_main_word_before_word = 0 # 前の単語に内容語があったか
    tmp_bigram_word = "" # bigramの一時保存
    # return value list
    token_main_words = []
    token_prefix_and_noun = []
    token_bigram = []

    mecab = MeCab.Tagger()
    mecab.parse("")
    node = mecab.parseToNode(text)
    while node:
        # リセット
        skip = 0
        # NG ワード確認
        if node.surface in NG_WORDS:
            is_main_word = 0
        else:
            # 内容語抽出
            if node.feature.split(",")[0] in ["名詞","動詞","形容詞"] and node.feature.split(",")[1] != "数":
                token_main_words.append(node.surface)
                is_main_word = 1
            else:
                is_main_word = 0
            # 名詞句抽出
            if node.feature.split(",")[0] == "接頭詞":
                is_prefix = 1
                tmp_noun_phrase += node.surface
            elif node.feature.split(",")[0] == "名詞":
                if cnt_head_noun >= 1 or is_prefix == 1:
                    tmp_noun_phrase += node.surface
                    cnt_head_noun += 1
                else:
                    cnt_head_noun += 1
                    tmp_noun_phrase += node.surface
                is_prefix = 0
            else:
                if cnt_head_noun > 1:
                    token_prefix_and_noun.append(tmp_noun_phrase)
                    is_prefix = 0
                    cnt_head_noun = 0
                    tmp_noun_phrase = ""
                else:
                    is_prefix = 0
                    cnt_head_noun = 0
                    tmp_noun_phrase = ""
            # 内容語のbigram抽出
            if tmp_bigram_word:
                if node.surface == "" or node.feature.split(",")[0] =="記号":
                    pass
                elif is_main_word_before_word == 1:
                    token_bigram.append(tmp_bigram_word + node.surface)
                elif is_main_word == 1:
                    token_bigram.append(tmp_bigram_word + node.surface)
                else:
                    pass
            # 最終処理-内容語のbigram抽出-
            tmp_bigram_word = node.surface
            if is_main_word == 1:
                is_main_word_before_word = 1
            else:
                is_main_word_before_word = 0
        node = node.next

    return token_main_words, token_prefix_and_noun, token_bigram

In [3]:
text="広告代理店に勤務するマサエさん（40歳）のお気に入り第二位は『100万回生きた猫』"
token_main_words, token_prefix_and_noun, token_bigram = extract_word(text)
print("token_main_words:")
print(token_main_words)
print("token_prefix_and_noun:")
print(token_prefix_and_noun)
print("token_bigram:")
print(token_bigram)

token_main_words:
['広告', '代理', '店', '勤務', 'する', 'マサエ', 'さん', '歳', 'お気に入り', '位', '回', '生き', '猫']
token_prefix_and_noun:
['広告代理店', 'マサエさん', '40歳', 'お気に入り第二位', '100万回']
token_bigram:
['広告代理', '代理店', '店に', 'に勤務', '勤務する', 'するマサエ', 'マサエさん', '40歳', 'のお気に入り', 'お気に入り第', '二位', '位は', '万回', '回生き', '生きた', 'た猫']


In [4]:
def word_dependency(text):
    """
    渡されたテキスト文に係り受けのペアを返す
    名詞-動詞
    """
    cabocha = CaboCha.Parser()
    tree = cabocha.parse(text)
    chunk_dic = {}
    chunk_id = 0
    for i in range(0, tree.size()):
        token = tree.token(i)
        if token.chunk:
            chunk_dic[chunk_id] = token.chunk
            chunk_id += 1
    dependency_token = []
    for chunk_id, chunk in chunk_dic.items():
        if chunk.link > 0:
            from_surface, from_feature =  word_dependency_get_word(tree, chunk)
            to_chunk = chunk_dic[chunk.link]
            to_surface, to_feature = word_dependency_get_word(tree, to_chunk)
            if from_feature != to_feature and from_feature != "形容詞" \
                and to_feature != "形容詞" and from_feature != "":
                dependency_token.append(from_surface + " " + to_surface)
    return dependency_token

def word_dependency_get_word(tree, chunk):
    """
    word_dependency中の処理
    係り受けとなっている単語の品詞を特定する
    """
    surface = ''
    feature = ''
    for i in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
        token = tree.token(i)
        features = token.feature.split(',')
        if features[0] == '名詞':
            surface += token.surface
            feature = '名詞'
        elif features[0] == '形容詞':
            surface += features[6]
            break
        elif features[0] == '動詞':
            surface += features[6]
            feature = '動詞'
            break
    return surface, feature

In [5]:
dependency_token = word_dependency(text)
print("dependency_token:")
print(dependency_token)

dependency_token:
['広告代理店 勤務する', '勤務する マサエさん', '100万回 生きる', '生きる 猫']


## classification.py

ライブラリの読み込み。  
log用のスクリプト設置

In [6]:
import logging
import csv
from datetime import datetime
from time import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

logging.basicConfig(filename='result.log',level=logging.INFO)

def log(content):
    time = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    print (time + ': ' + content)
    logging.info(time + ': ' + content)

サンプル用にニュースサイトのテキストデータを取得する  
https\://www.rondhuit.com/download.html  
同一フォルダ内に解凍したデータを置く  

今回は、「peachy」「sports-watch」の２カテゴリを調査  
各200文書を取得  

==peachy==  
各記事ファイルにはクリエイティブ・コモンズライセンス「表示 – 改変禁止」が適用されます。 クレジット表示についてはニュースカテゴリにより異なるため、ダウンロードしたファイルを展開したサブディレクトリにあるそれぞれの LICENSE.txt をご覧ください。 livedoor はNHN Japan株式会社の登録商標です。  
==sports-watch==  
このディレクトリにあるすべての記事ファイルには、クリエイティブ・コモンズライセンス「表示 - 改変禁止」（http\://creativecommons.org/licenses/by-nd/2.1/jp/）が適用されます。原著作者のクレジットを表示し、ニュース記事の改変をしないことを条件に、記事全文を自由に転載・引用が可能です。
このディレクトリの記事ファイル内容の提供元：Sports Watch http\://news.livedoor.com/category/vender/208/

In [7]:
import os

TARGET_CATEGORY=["peachy","sports-watch"]

def read_news():
    """
    サンプルデータの読み込み
    """
    with open("example.csv","w") as wf:
        writer = csv.writer(wf)
        for category in TARGET_CATEGORY:
            files = os.listdir(os.path.join('text',category))
            cnt=0
            for file in files:
                contents =""
                tokens=[]
                if file=="LICENSE.txt" or cnt>200:
                    continue
                with open(os.path.join("text",category,file),"r") as rf:
                    reader = csv.reader(rf)
                    for row in reader:
                        if not row:
                            continue
                        contents += row[0]
                        token_main_words, token_prefix_and_noun, token_bigram = extract_word(row[0])
                        token = token_main_words + token_prefix_and_noun
                        tokens.extend(token)
                    concated_tokens=" ".join(tokens)
                    writer.writerow([contents, concated_tokens, category])

In [8]:
# データの取得
read_news()

先ほど抽出したニュースデータを読み込む

In [9]:
def load_dataset():
    """
    import dataset
    ## example.csv
    row#1: raw contents
    row#2: labels data
    row#3: tokenized contents (tfidf:separeted by space)
    """
    contents=[]
    labels=[]
    tokens=[]
    with open("example.csv") as f:
        reader = csv.reader(f)
        header = next(reader)
        for row in reader:
            contents.append(row[0])
            tokens.append(row[1])
            labels.append(row[2])

    return contents, tokens, labels

In [10]:
contents, tokens, labels = load_dataset()

In [11]:
print("labels:")
print(labels[0])
print()
print("contents:")
print(contents[0])
print()
print("tokens:")
print(tokens[0])

labels:
peachy

contents:
http://news.livedoor.com/article/detail/6732572/2012-07-07T18:00:00+0900−31キロを実現したダイエット法／美肌を作る30秒マッサージなど−【ビューティー】週間ランキング美しさを追求するのは、女子の“永遠の課題”と言っても過言ではありません。「ファッション・ビューティ」カテゴリの中から、Peachyのアプリで先週（2012年6月28日〜7月4日）最も読まれた記事TOP5をお届けします！第1位：あびる優、体重を公表　スレンダーボディの秘訣明かすタレントのあびる優が、オフィシャルブログにてファンから寄せられた質問に回答。スレンダーボディをキープする秘訣を明かした。第2位：女性のムダ毛にガッカリ…7割の男性は女性のムダ毛に気づいている！薄着になり、水着が店頭に並び、女性がダイエットを意識する夏前のこの季節、ダイエット以外にも意識しておきたいことがある。それが「ムダ毛処理」だ第3位：藤井リナ、すっぴん美肌の秘訣を伝授雑誌「VoCE」8月号に、人気モデルの藤井リナが登場した。28歳を目前にした今、これまでの“カワイイ”に留まらずエロさ、毒、愛らしさを秘めた大人の魅力を放ち始めた藤井リナ。そんな彼女の記念すべき「VoCE」初インタビューでは、安室奈美恵ら数多くの有名人を手がけるヘアメイク・中野明海さんとの初セッションが実現。スキンケア、スタイルキープ法、ストレス解消法など、ビューティーにまつわる様々なこだわりを明かしている。第4位：一日の美肌の鍵は「朝の30秒マッサージ」だった！きゃ〜、美容に時間をかけていたら遅刻しちゃう！　一分一秒でも惜しい、忙しい朝、みなさんはどんなスキンケアをしていますか？　また、朝のスキンケアは、夜のスキンケアと比べて“手をかけるレベル”はいかがでしょうか？株式会社ネオマーケティングが2012年6月に20歳〜30歳代の女性1第5位：−31kgの重量級痩せに成功！そのダイエット法とは雑誌「Popteen」8月号の特集「ガチやせDIET総選挙2012・夏」では、−31kgを筆頭とした最強ヤセ読者たちのダイエット術を大解剖。まるで別人へと生まれ変わった彼女たちの、壮絶な努力の物語が語られている。以上、先週の「ビューティ」カテゴリの人気記事

In [12]:
def format_labels(labels):
    """
    教師ラベルをシリアル変換
    category_list:カテゴリの位置に対応した名称を保存
    """
    # カテゴリ名取得、シリアル変換
    l_encoder = LabelEncoder()
    l_encoder.fit(labels)
    category_list = list(l_encoder.classes_)
    serialized_labels = l_encoder.transform(labels)

    return serialized_labels, category_list

ラベルデータのシリアライズ化

In [13]:
modified_labels, category_list = format_labels(labels)

In [14]:
print("modified_labels:")
print(modified_labels)
print()
print("category_list:")
print(category_list)

modified_labels:
[0 0 0 ..., 1 1 1]

category_list:
['peachy', 'sports-watch']


ラベルの情報  
0:peachy  
1:sports-watch

In [15]:
def execute_model(pipeline, params, contents, tokens, label, category_list):
    """
    classification 実行
    """
    log("-------------------------")
    log("----- Score  Report -----")
    log("-------------------------")
    t0 = time()
    train_x, test_x, train_y, test_y = train_test_split(tokens, label, test_size=0.2, random_state=22)
    train_x_raw, test_x_raw, train_y_raw, test_y_raw = train_test_split(contents, label, test_size=0.2, random_state=22)
    clf = GridSearchCV(pipeline, params)
    clf.fit(train_x,train_y)
    log("done in %0.3fs" % (time() - t0))
    log("score %f" % (clf.score(train_x,train_y)))
    
    # グリッドサーチをしないときは以下の部分をコメントアウトする
    log("Best parameters set:")
    best_parameters = clf.best_estimator_.get_params()
    for param_name in sorted(params.keys()):
        log("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    pred = clf.predict(test_x)
    confidence = clf.predict_proba(test_x)
    # スコア計算
    F1_score = f1_score(test_y, pred, average="weighted")
    accur = accuracy_score(test_y, pred)
    log("F1_score: %f" % F1_score)
    log("ACCUR: %f" % accur)
    log(classification_report(test_y, pred, target_names=category_list))
    log("done in %0.3fs" % (time() - t0))
    # joblib.dump(clf, 'clf.pkl')

    with open("predict_result.csv","w") as f:
        writer = csv.writer(f)
        for (x,y,p,c) in zip(test_x_raw,test_y,pred,confidence):
            writer.writerow([x,y,p,max(c)])

In [16]:
# グリッドサーチ用のパラメーター
params={
'lr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
'vectorizer__norm': [None, 'l2'],
}
pipeline = Pipeline([
('vectorizer', TfidfVectorizer()),
('lr', LogisticRegression(class_weight = "balanced"))
])

In [17]:
execute_model(pipeline, params, contents, tokens, modified_labels, category_list)

2017/08/31 13:45:21: -------------------------
2017/08/31 13:45:21: ----- Score  Report -----
2017/08/31 13:45:21: -------------------------
2017/08/31 13:46:00: done in 38.997s
2017/08/31 13:46:01: score 1.000000
2017/08/31 13:46:01: Best parameters set:
2017/08/31 13:46:01: 	lr__C: 1000
2017/08/31 13:46:01: 	vectorizer__norm: 'l2'
2017/08/31 13:46:01: F1_score: 0.994268
2017/08/31 13:46:01: ACCUR: 0.994269
2017/08/31 13:46:01:               precision    recall  f1-score   support

      peachy       0.99      1.00      0.99       181
sports-watch       1.00      0.99      0.99       168

 avg / total       0.99      0.99      0.99       349

2017/08/31 13:46:01: done in 39.780s


In [18]:
import pandas as pd
df = pd.read_csv("predict_result.csv",names=("contents","labels","predict","confidence"))
#見やすいように置換
mapping={0:'peachy', 1:'sports-watch'}
df[["labels","predict"]] = df[["labels","predict"]].replace(mapping)

In [19]:
df.head(10)

,contents,labels,predict,confidence
0,http://news.livedoor.com/article/detail/474526...,sports-watch,sports-watch,0.988596
1,http://news.livedoor.com/article/detail/591090...,peachy,peachy,0.999831
2,http://news.livedoor.com/article/detail/491990...,sports-watch,sports-watch,0.996417
3,http://news.livedoor.com/article/detail/609003...,sports-watch,sports-watch,0.999916
4,http://news.livedoor.com/article/detail/668482...,peachy,peachy,0.999915
5,http://news.livedoor.com/article/detail/470367...,sports-watch,sports-watch,0.999511
6,http://news.livedoor.com/article/detail/496061...,sports-watch,sports-watch,0.995080
7,http://news.livedoor.com/article/detail/565495...,sports-watch,sports-watch,0.995340
8,http://news.livedoor.com/article/detail/690798...,peachy,peachy,0.999802
9,http://news.livedoor.com/article/detail/555669...,sports-watch,sports-watch,0.999216


In [20]:
print("labels:")
print(df.iloc[1]["labels"])
print("predict:")
print(df.iloc[1]["predict"])
print()
print("contents:")
print(df.iloc[1]["contents"])

labels:
peachy
predict:
peachy

contents:
http://news.livedoor.com/article/detail/5910900/2011-10-04T14:37:00+0900【終了しました】ムリをせず幸せになりたいオトナ女子へ。カウンセラー・五百田達成著『心のゆるめかた』を5名様にプレゼント「毎日充実しているけど、このままでいいか考えてしまう時がある」「特に理由もないのに、将来が漠然と不安」など、なんとなくモヤモヤするときはありませんか?　そのままズルズルと落ち込んでしまい、ブルーな気分から抜けだせないという経験は誰しも持っているのではないでしょうか。その原因のひとつとして、恋愛や仕事を成功させたいあまり、「頑張りすぎて空回っている」状態が挙げられます。焦りやプレッシャー、不安などから解放されて、もっと楽に毎日を過ごしたいですよね。そんなオトナ女子の心のお疲れモードを解消したいときにお勧めしたいのが、メディアでも話題の”オトナ女子カウンセラー”五百田達成が著した『心のゆるめかた』です。この本では、モヤモヤする気持ちは心が凝り固まっているからだとし、凝った心をほぐすためのヒントが紹介されています。すべての女子の胸の奥には、「幸せになる本能」が眠っています。・「理屈」よりも「気持ち」や「直感」を大事にしてる？・将来のことを考えすぎてない？・焦る気持ちでいっぱいいっぱいになってない？・ちゃんと自分を褒めてあげてる？「我慢してないのに愛される。頑張ってないのに夢が叶う」この本を読めば、そんな魔法のようなことが起きるのだそうです。そして、ゆるんだ心の明るいオーラは、人を惹きつける力があるのだとか。実際に読んだ方からは、・ずっと好きだった人からデートに誘われました（20代・営業職）・『だいじょうぶ』と言ってもらえて涙がこぼれました（40代・教師）・震災以降のモヤモヤがすーっと晴れました（30代・SE）などの嬉しい声が寄せられています。今回Peachyでは、この『心のゆるめかた』を5名様にプレゼントいたします。この機会に心の凝りをほぐして、頑張らずに幸せになれる心のゆるめ方をレッスンしてみてはいかがでしょうか。五百田達成著『心のゆるめかた』を5名様にプレゼント【賞品・応募数】五百田達成著『心のゆるめかた』（中経出版） / 5名